### `MaxPool2D` code

In [15]:
import numpy as np
import torch # needed for tests
from tqdm import tqdm # needed for tests

In [16]:
class MaxPool2D:
    ''' Computes maxpool given the input parameters '''
    
    ''' * The class implementation will be along the lines of torch.nn.MaxPool2d in order to 
          enable comparison of this NumPy only implementation and seamless testing
    '''
    '''
        TODO:
        * Replace `torch.round()` with `np.allclose()` for tests
        * Optimizing code
    '''
    
    def __init__(
        self, 
        # in_channels, 
        # out_channels, 
        kernel_size, 
        padding = (0, 0), 
        stride = (1, 1), 
        dilation = (1, 1), 
        # groups = 1, 
        # bias = True, 
        # padding_mode = 'zeros', 
        # device = None, 
        # dtype = None, 
        verbose = False
        ):
        super(MaxPool2D, self).__init__()
        
        ''' mandatory parameters '''
        # self.in_channels = in_channels
        # self.out_channels = out_channels
        if isinstance(kernel_size, tuple):
            self.kernel_size = kernel_size
        elif isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        else:
            raise Exception('invalid input parameters: kernel_size should either be an int or a tuple')
        
        ''' optional parameters '''
        if isinstance(padding, str):
            if padding == 'valid':
                self.padding = (0, 0)
            elif padding == 'same':
                raise Exception("invalid input parameters: padding = 'same' not yet supported")
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, tuple):
            if padding[0] >= 0 and padding[1] >= 0:
                self.padding = padding
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, int):
            if padding >= 0:
                self.padding = (padding, padding)
            else:
                raise Exception('invalid input parameters: padding is not valid')
        else:
            raise Exception('invalid input parametersL padding should be either an int or a tuple')
        if isinstance(stride, tuple):
            if stride[0] >= 1 and stride[1] >= 1:
                self.stride = stride
            else:
                raise Exception('invalid input parameters: stride is not valid')
        elif isinstance(stride, int):
            if stride >= 1:
                self.stride = (stride, stride)
            else:
                raise Exception('invalid input parameters: stride is not valid')
        else:
            raise Exception('invalid input parameters: stride should be either an int or a tuple')
        if isinstance(dilation, tuple):
            if dilation[0] >= 1 and dilation[1] >= 1:
                self.dilation = dilation
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        elif isinstance(dilation, int):
            if dilation >= 1:
                self.dilation = (dilation, dilation)
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        else:
            raise Exception('invalid input parameters: dilation should be either an int or a tuple')
        # self.groups = groups
        
        ''' optional parameters (dummy, yet to be implemented)'''
        # self.bias = bias
        # self.padding_mode = padding_mode
        # self.device = device
        # self.dtype = dtype
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        # self.verboseprint('in_channels: {}, out_channels: {}, kernel_size: {}'.format(self.in_channels, self.out_channels, self.kernel_size))
        self.verboseprint('kernel_size: {}'.format(self.kernel_size))
        self.verboseprint('padding: {}, stride: {}, dilation factor: {}'.format(self.padding, self.stride, self.dilation))
        # self.verboseprint('groups: {}, bias: {}, padding_mode: {}, device: {}, dtype: {}'.format(self.groups, self.bias, self.padding_mode, self.device, self.dtype))
        # self.verboseprint('groups: {}'.format(self.groups))
        self.verboseprint('\n')
    
    def forward(self, _input):
        ''' forward pass to perform convolution '''
        
        ''' do error checking '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        if _input_h + 2 * self.padding[0] < self.dilation[0] * (self.kernel_size[0] - 1) + 1: # check if (dilated) ker_h is valid
            raise Exception('invalid input parameters: kernel height is larger than input height')
        if _input_w + 2 * self.padding[1] < self.dilation[1] * (self.kernel_size[1] - 1) + 1: # check if (dilated) ker_w is valid
            raise Exception('invalid input parameters: kernel width is larger than input width')
        if ((_input_h + 2 * self.padding[0] - (self.dilation[0] * (self.kernel_size[0] - 1) + 1)) / self.stride[0]) + 1 < 0: # check if out_h is valid
            raise Exception('invalid input parameters: output height is negative')
        if ((_input_w + 2 * self.padding[1] - (self.dilation[1] * (self.kernel_size[1] - 1) + 1)) / self.stride[1]) + 1 < 0: # check if out_w is valid
            raise Exception('invalid input parameters: output width is negative')
        # if  self.in_channels % self.groups != 0: # check if groups is valid
            # raise Exception('invalid input parameters: input channels is not divisible by groups')
        # if self.out_channels % self.groups != 0: # check if groups is valid
            # raise Exception('invalid input parametes: output channels is not divisible by groups')
         
        ''' add zero padding based on the input parameters '''
        if self.padding != (0, 0):
            _input = np.array([[np.pad(channel, ((self.padding[0], self.padding[0]), (self.padding[1], self.padding[1])), 'constant', constant_values = -np.inf) for channel in batch] for batch in _input])    
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(_input.shape[0], _input.shape[1], _input.shape[2], _input.shape[3]))
            self.verboseprint(_input)
            self.verboseprint('\n')
        
        ''' use the provided kernels or create random kernels based on the input kernel parameters '''
        '''
        if kernels is not None:
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        else:
            kernels = []
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
            for k in range(self.out_channels):
                kernel = np.random.rand(int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]) # define a random kernel based on the kernel parameters
                kernels.append(kernel)
                self.verboseprint('kernel {}'.format(k))
                self.verboseprint(kernel)
            self.verboseprint('\n')
        '''
        
        ''' dilate a kernel '''
        dil_ker_h = self.dilation[0] * (self.kernel_size[0] - 1) + 1
        dil_ker_w = self.dilation[1] * (self.kernel_size[1] - 1) + 1
        '''
        dil_kernels = []
        for kernel in kernels:
            dil_kernel = []
            for channel in kernel:
                dil_channel = np.zeros((dil_ker_h, dil_ker_w))
                for row in range(len(channel)):
                    for col in range(len(channel[0])):
                        dil_channel[self.dilation[0] * row][self.dilation[1] * col] = channel[row][col] # check if the indices 0 and 1 need to be swapped
                dil_kernel.append(dil_channel.tolist())
            dil_kernels.append(dil_kernel)
        '''
        # kernels, self.kernel_size = dil_kernels, (dil_ker_h, dil_ker_w)
        self.kernel_size = (dil_ker_h, dil_ker_w)
        self.verboseprint('*** dilated kernels ***')
        # self.verboseprint('kernels: {}, dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, self.dilation, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        self.verboseprint('dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.dilation, int(_input_c), self.kernel_size[0], self.kernel_size[1]))
        '''
        for k in range(self.out_channels):
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernels[k])
        '''
        self.verboseprint('\n')
        
        ''' compute output volume from the input and kernel parameters '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        out_n = int(_input_n)
        out_c = int(_input_c)
        out_h = int((_input_h - self.kernel_size[0]) / self.stride[0]) + 1
        out_w = int((_input_w - self.kernel_size[1]) / self.stride[1]) + 1
        output = np.zeros([out_n, out_c, out_h, out_w]) # may need to initialize to -np.inf
        
        ''' parse through every element of the output and compute the convolution value for that element '''
        for b in range(out_n):
            for k in range(out_c):
                for h in range(out_h):
                    for w in range(out_w):
                        # convolve kernel over the input slices
                        self.verboseprint('kernel indices, image indices')
                        self.verboseprint('[n, c, h, w]', '[n, c, h, w]')
                        convol_sum = -np.inf
                        # ker_c = 1
                        ker_h = self.kernel_size[0]
                        ker_w = self.kernel_size[1]
                        # for c_ker in range(ker_c):
                        for h_ker in range(ker_h):
                            for w_ker in range(ker_w):
                                self.verboseprint([b, k, h_ker + self.stride[0] * h, w_ker + self.stride[1] * w])
                                if _input[b][k][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w] > convol_sum: # works for all 1 <= `groups` <= in_channels
                                    convol_sum = _input[b][k][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w]

                                # self.verboseprint([k, c_ker, h_ker, w_ker], [b, c_ker + (ker_c * int(np.floor(k / _input_c))), h_ker + self.stride[0] * h, w_ker + self.stride[1] * w])
                                # if _input[b][c_ker + (ker_c * int(np.floor(k / _input_c)))][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w] > convol_sum: # works for all 1 <= `groups` <= in_channels
                                    # convol_sum = _input[b][c_ker + (ker_c * int(np.floor(k / _input_c)))][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w]
                                # self.verboseprint([k, c_ker, h_ker, w_ker], [b, c_ker + (ker_c * int(np.floor(k / (self.out_channels / self.groups)))), h_ker + self.stride[0] * h, w_ker + self.stride[1] * w])
                                # convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker + (ker_c * int(np.floor(k / (self.out_channels / self.groups))))][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w] # works for all 1 <= `groups` <= in_channels
                        self.verboseprint('\n')
                        output[b, k, h, w] += convol_sum
        self.verboseprint('*** MaxPool2D output ***')
        output_shape = output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((out_n, out_c, out_h, out_w) == output_shape)
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test (random kernel, random input)

In [114]:
in_channels = 15 #2 # input channels
# out_channels = 4 # output channels
kernel_size = (7, 2) #(2, 2) # kernel size

padding = (1, 1) #(0, 0) # padding (optional)
stride = (4, 4) #(2, 2) # stride (optional)
dilation = (1, 9) #(1, 1) # dilation factor (optional)
# groups = 2 # groups (optional)

in_batches = 1 #1 # input batches
in_h = 20 #4 # input height
in_w = 20 #4 # input weight

np.random.seed(30)
_input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
# kernels = []
# for k in range(out_channels):
    # kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
    # kernels.append(kernel)

In [115]:
# get MaxPool2D output with the random inputs

maxpool2d = MaxPool2D(kernel_size, stride = stride, padding = padding, dilation = dilation) # call an instance of the class with the input parameters 
_output = maxpool2d.forward(_input) # perform convolution
print("*** MaxPool2D output ***")
print(_output)

*** MaxPool2D output ***
[[[[0.99310313 0.99310313 0.95691278 0.99419368]
   [0.99310313 0.99310313 0.98865854 0.98339226]
   [0.99741096 0.98313394 0.98444799 0.98742331]
   [0.99894214 0.99894214 0.99894214 0.98742331]]

  [[0.99454453 0.99454453 0.99454453 0.97349741]
   [0.99454453 0.99454453 0.99454453 0.99443237]
   [0.97598299 0.97357177 0.94153434 0.99047825]
   [0.97598299 0.99471545 0.99471545 0.99047825]]

  [[0.97291192 0.98857684 0.99708791 0.99708791]
   [0.98003362 0.98916951 0.99708791 0.99708791]
   [0.98003362 0.98916951 0.98916951 0.96189053]
   [0.99974348 0.99974348 0.98088865 0.96796626]]

  [[0.97587393 0.99921808 0.99921808 0.99444064]
   [0.98700251 0.98700251 0.96643368 0.95632998]
   [0.98700251 0.98700251 0.95790703 0.95790703]
   [0.96080253 0.95988199 0.95790703 0.96593383]]

  [[0.99659986 0.98579669 0.99605204 0.99605204]
   [0.9765062  0.9765062  0.99605204 0.99605204]
   [0.99282703 0.966368   0.98760292 0.98760292]
   [0.99282703 0.98267004 0.98267004

In [116]:
# get PyTorch output with the same random inputs as above

x = torch.DoubleTensor(_input)
# weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
m = torch.nn.MaxPool2d(kernel_size, stride = stride, padding = padding, dilation = dilation)
output = m(x)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[0.6326, 0.9931, 0.9569, 0.6379],
          [0.9887, 0.9931, 0.9569, 0.7702],
          [0.9651, 0.8978, 0.9831, 0.7702],
          [0.9989, 0.9343, 0.9831, 0.7174]],

         [[0.9604, 0.9412, 0.9945, 0.8164],
          [0.9837, 0.9412, 0.9945, 0.8164],
          [0.9373, 0.9415, 0.8484, 0.8456],
          [0.9600, 0.9079, 0.9435, 0.8456]],

         [[0.6688, 0.9886, 0.9729, 0.7704],
          [0.9222, 0.9412, 0.9729, 0.7884],
          [0.9469, 0.9296, 0.9608, 0.9360],
          [0.9469, 0.9511, 0.9608, 0.9680]],

         [[0.6560, 0.8000, 0.9759, 0.9554],
          [0.9377, 0.7640, 0.9231, 0.8397],
          [0.9377, 0.9579, 0.8579, 0.7814],
          [0.7448, 0.9579, 0.8579, 0.9078]],

         [[0.7505, 0.9027, 0.9486, 0.8230],
          [0.9219, 0.9027, 0.9271, 0.8641],
          [0.9219, 0.8287, 0.8203, 0.9249],
          [0.9365, 0.8481, 0.9827, 0.9249]],

         [[0.8813, 0.9504, 0.9810, 0.8055],
          [0.8813, 0.9504, 0.9285, 0.8530],

In [113]:
# compare outputs of Conv2D and PyTorch
print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

False


### Extensive tests (random kernel, random input)

In [7]:
def valid_params(num_tests):
    ''' generates `num_tests` number of valid input and kernel parameters '''
    
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_channels = np.random.randint(20) + 1 # input channels
        # out_channels = np.random.randint(20) + 1 # output channels
        
        kernel_h = np.random.randint(20) + 1
        kernel_w = np.random.randint(20) + 1
        kernel_size = (kernel_h, kernel_w) # kernel size
        
        padding_h = np.random.randint(10) + 1
        padding_w = np.random.randint(10) + 1
        padding = (padding_h, padding_w) # padding (optional)
        stride_h = np.random.randint(5) + 1
        stride_w = np.random.randint(5) + 1
        stride = (stride_h, stride_w) # stride (optional)
        dilation_h = np.random.randint(10) + 1
        dilation_w = np.random.randint(10) + 1
        dilation = (dilation_h, dilation_w) # dilation factor (optional)
        # groups = np.random.randint(in_channels) + 1 # groups (optional)
        
        in_batches = np.random.randint(5) + 1 # input batches
        in_h = np.random.randint(30) + 5 # input height
        in_w = np.random.randint(30) + 5 # input weight
    
        ker_h_flag, ker_w_flag, out_h_flag, out_w_flag, pad_h_flag, pad_w_flag = True, True, True, True, True, True
        
        if in_h + 2 * padding_h < dilation_h * (kernel_h - 1) + 1: # check if (dilated) ker_h is valid
            ker_h_flag = False
        if in_w + 2 * padding_w < dilation_w * (kernel_w - 1) + 1: # check if (dilated) ker_w is valid
            ker_w_flag = False
        if ((in_h + 2 * padding_h - (dilation_h * (kernel_h - 1) + 1)) / stride_h) + 1 < 0: # check if out_h is valid
            out_h_flag = False
        if ((in_w + 2 * padding_w - (dilation_w * (kernel_w - 1) + 1)) / stride_w) + 1 < 0: # check if out_w is valid
            out_w_flag = False
        if padding_h > kernel_h // 2:
            pad_h_flag = False
        if padding_w > kernel_w // 2:
            pad_w_flag = False
        # if in_channels % groups != 0: # check if groups is valid
            # in_group_flag = False
        # if out_channels % groups != 0: # check if groups is valid
            # out_group_flag = False
        
        if ker_h_flag and ker_w_flag and out_h_flag and out_w_flag and pad_h_flag and pad_w_flag:
            params_list.append({'in_channels': in_channels, 'kernel_size': kernel_size,
                          'padding': padding, 'stride': stride, 'dilation': dilation, 'in_batches': in_batches,
                          'in_h': in_h, 'in_w': in_w})
            sample_count += 1
    return params_list

In [8]:
def run_tests(num_tests):
    ''' sweep different input parameters and test by comparing outputs of Conv2D and PyTorch '''
    
    num_passed = 0
    params_list = valid_params(num_tests)
    print('Number of tests: {}\n\n'.format(len(params_list)))

    for i, params in enumerate(tqdm(params_list)):
        print('Test: {}\nParams: {}'.format(i, params))
        in_channels = params['in_channels'] # input channels
        # out_channels = params['out_channels'] # output channels
        kernel_size = params['kernel_size'] # kernel size

        padding = params['padding'] # padding (optional)
        stride = params['stride'] # stride (optional)
        dilation = params['dilation'] # dilation factor (optional)
        # groups = params['groups'] # groups (optional)

        in_batches = params['in_batches'] # input batches
        in_h = params['in_h'] # input height
        in_w = params['in_w'] # input weight

        _input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
        # kernels = []
        # for k in range(out_channels):
            # kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
            # kernels.append(kernel)

        try:
            # get MaxPool2D output with the random inputs
            maxpool2d = MaxPool2D(kernel_size, stride = stride, padding = padding, dilation = dilation) # call an instance of the class with the input parameters 
            _output = maxpool2d.forward(_input) # perform convolution

            # get PyTorch output with the same random inputs as above
            x = torch.DoubleTensor(_input)
            m = torch.nn.MaxPool2d(kernel_size, stride = stride, padding = padding, dilation = dilation)
            output = m(x)

        except Exception as e:
            print(e)
            print('Result: False\n\n') # treating exception as a failed test
            continue

        # compare outputs of MaxPool2D and PyTorch
        result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
        print('Result: {}\n\n'.format(result))
        if result:
            num_passed += 1

    print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))


In [10]:
num_tests = 100
run_tests(num_tests)

Number of tests: 100




  3%|█▎                                         | 3/100 [00:00<00:03, 26.69it/s]

Test: 0
Params: {'in_channels': 2, 'kernel_size': (7, 12), 'padding': (2, 1), 'stride': (4, 4), 'dilation': (2, 3), 'in_batches': 4, 'in_h': 12, 'in_w': 33}
Result: True


Test: 1
Params: {'in_channels': 14, 'kernel_size': (20, 2), 'padding': (5, 1), 'stride': (2, 2), 'dilation': (1, 6), 'in_batches': 3, 'in_h': 19, 'in_w': 11}
Result: True


Test: 2
Params: {'in_channels': 17, 'kernel_size': (15, 16), 'padding': (5, 6), 'stride': (5, 5), 'dilation': (1, 1), 'in_batches': 1, 'in_h': 19, 'in_w': 6}
Result: True


Test: 3
Params: {'in_channels': 18, 'kernel_size': (16, 11), 'padding': (1, 4), 'stride': (3, 4), 'dilation': (1, 1), 'in_batches': 4, 'in_h': 17, 'in_w': 11}
Result: True


Test: 4
Params: {'in_channels': 16, 'kernel_size': (15, 16), 'padding': (2, 8), 'stride': (2, 1), 'dilation': (2, 1), 'in_batches': 5, 'in_h': 31, 'in_w': 16}


  8%|███▍                                       | 8/100 [00:02<00:26,  3.50it/s]

Result: True


Test: 5
Params: {'in_channels': 2, 'kernel_size': (11, 8), 'padding': (5, 4), 'stride': (5, 4), 'dilation': (3, 3), 'in_batches': 1, 'in_h': 21, 'in_w': 15}
Result: True


Test: 6
Params: {'in_channels': 5, 'kernel_size': (14, 10), 'padding': (1, 5), 'stride': (2, 1), 'dilation': (1, 4), 'in_batches': 1, 'in_h': 29, 'in_w': 28}
Result: True


Test: 7
Params: {'in_channels': 4, 'kernel_size': (12, 15), 'padding': (6, 3), 'stride': (3, 5), 'dilation': (2, 1), 'in_batches': 3, 'in_h': 17, 'in_w': 32}
Result: True


Test: 8
Params: {'in_channels': 20, 'kernel_size': (10, 18), 'padding': (1, 3), 'stride': (4, 5), 'dilation': (1, 2), 'in_batches': 1, 'in_h': 10, 'in_w': 29}
Result: True


Test: 9
Params: {'in_channels': 19, 'kernel_size': (3, 10), 'padding': (1, 4), 'stride': (1, 1), 'dilation': (6, 1), 'in_batches': 4, 'in_h': 33, 'in_w': 8}


...

Result: True


Test: 90
Params: {'in_channels': 6, 'kernel_size': (9, 5), 'padding': (3, 2), 'stride': (1, 5), 'dilation': (1, 1), 'in_batches': 2, 'in_h': 14, 'in_w': 26}
Result: True


Test: 91
Params: {'in_channels': 9, 'kernel_size': (10, 7), 'padding': (1, 3), 'stride': (3, 2), 'dilation': (2, 1), 'in_batches': 1, 'in_h': 19, 'in_w': 5}
Result: True


Test: 92
Params: {'in_channels': 6, 'kernel_size': (14, 5), 'padding': (6, 2), 'stride': (2, 2), 'dilation': (1, 2), 'in_batches': 3, 'in_h': 11, 'in_w': 23}
Result: True


Test: 93
Params: {'in_channels': 2, 'kernel_size': (18, 10), 'padding': (8, 5), 'stride': (1, 2), 'dilation': (1, 4), 'in_batches': 4, 'in_h': 9, 'in_w': 28}
Result: True


Test: 94
Params: {'in_channels': 13, 'kernel_size': (14, 14), 'padding': (4, 4), 'stride': (1, 4), 'dilation': (1, 2), 'in_batches': 5, 'in_h': 18, 'in_w': 32}


 96%|████████████████████████████████████████▎ | 96/100 [00:31<00:00,  4.45it/s]

Result: True


Test: 95
Params: {'in_channels': 9, 'kernel_size': (8, 18), 'padding': (3, 6), 'stride': (1, 3), 'dilation': (3, 1), 'in_batches': 1, 'in_h': 26, 'in_w': 22}
Result: True


Test: 96
Params: {'in_channels': 20, 'kernel_size': (16, 7), 'padding': (8, 2), 'stride': (5, 1), 'dilation': (1, 2), 'in_batches': 1, 'in_h': 34, 'in_w': 34}


100%|█████████████████████████████████████████| 100/100 [00:32<00:00,  3.12it/s]

Result: True


Test: 97
Params: {'in_channels': 15, 'kernel_size': (13, 11), 'padding': (5, 2), 'stride': (4, 5), 'dilation': (1, 1), 'in_batches': 5, 'in_h': 29, 'in_w': 9}
Result: True


Test: 98
Params: {'in_channels': 11, 'kernel_size': (19, 15), 'padding': (7, 1), 'stride': (4, 4), 'dilation': (1, 1), 'in_batches': 3, 'in_h': 12, 'in_w': 17}
Result: True


Test: 99
Params: {'in_channels': 3, 'kernel_size': (13, 14), 'padding': (5, 1), 'stride': (4, 4), 'dilation': (2, 2), 'in_batches': 1, 'in_h': 33, 'in_w': 33}
Result: True


98 out of 100 (98.0%) tests passed
